In [1]:
from IPython.display import display, HTML
import pandas as pd
import sqlite3
from sqlite3 import Error

def create_connection(db_file, delete_db=False):
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql):
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def execute_sql_statement(sql_statement, conn):
    cur = conn.cursor()
    cur.execute(sql_statement)

    rows = cur.fetchall()

    return rows

conn = create_connection('la_city_crime.db', delete_db=True)

In [2]:
#Read dataset

df = pd.read_csv("Arrest_data.csv")

df = df.dropna()
print(len(df))
print(df.columns)

/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (21,23) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


414390
Index(['Report ID', 'Report Type', 'Arrest Date', 'Time', 'Area ID',
       'Area Name', 'Reporting District', 'Age', 'Sex Code', 'Descent Code',
       'Charge Group Code', 'Charge Group Description', 'Arrest Type Code',
       'Charge', 'Charge Description', 'Disposition Description', 'Address',
       'Cross Street', 'LAT', 'LON', 'Location', 'Booking Date',
       'Booking Time', 'Booking Location', 'Booking Location Code'],
      dtype='object')


In [3]:
sql_charge_detals = "CREATE TABLE ChargeDetails(ChargeGroupCode INTEGER NOT NULL PRIMARY KEY, ChargeGroupDesc TEXT NOT NULL, Charge TEXT NOT NULL,ChargeDesc TEXT NOT NULL);"

create_table(conn, sql_charge_detals)

rows = []
charge_group_code = []
charge_group_desc = []
charge_code = []
charge_desc = []

df_charge_data = df[['Charge Group Code', 'Charge Group Description', 'Charge', 'Charge Description']]
print(df_charge_data.head(), len(df_charge_data), df_charge_data.index)

#count = 0
for i in df_charge_data.index[0:10000]:
    count += 1
    #print(count)
    if(int(df_charge_data['Charge Group Code'][i]) not in charge_group_code):
        charge_group_code += [int(df_charge_data['Charge Group Code'][i])]
        charge_group_desc += [df_charge_data['Charge Group Description'][i]]
        charge_code += [df_charge_data['Charge'][i] + ":"]
        charge_desc += [df_charge_data['Charge Description'][i] + ":"]
    else:
        index = charge_group_code.index(int(df_charge_data['Charge Group Code'][i]))
        if(df_charge_data['Charge'][i] not in charge_code[index]):
            charge_code[index] += df_charge_data['Charge'][i] + ":"
            charge_desc[index] += df_charge_data['Charge Description'][i] + ":"

for i in range(0, len(charge_group_code)):
    rows += [(int(charge_group_code[i]), charge_group_desc[i], charge_code[i], charge_desc[i],)]

#print(charge_group_code)
    
#print(len(charge_group_code), len(list(set(charge_group_code))))
#print(rows)
sql = """INSERT INTO ChargeDetails VALUES (?,?,?,?);"""
cur = conn.cursor()
cur.executemany(sql, rows)



      Charge Group Code Charge Group Description       Charge  \
331                 5.0                 Burglary        459PC   
673                 6.0                  Larceny        666PC   
784                 5.0                 Burglary        459PC   
2256               10.0       Fraud/Embezzlement  10980(C)2WI   
3108                7.0            Vehicle Theft   10851(A)VC   

                             Charge Description  
331                                    BURGLARY  
673    PETTY THEFT W/SPECIFIED PRIOR CONVICTION  
784                                    BURGLARY  
2256  OBTAIN/ETC AID BY FALSE STMENT/ETC > $400  
3108           TAKE VEHICLE W/O OWNER'S CONSENT   414390 Int64Index([    331,     673,     784,    2256,    3108,    4675,    4693,
               4922,    5175,   44678,
            ...
            1322791, 1322792, 1322793, 1322795, 1322798, 1322805, 1322808,
            1322814, 1322817, 1322819],
           dtype='int64', length=414390)


In [4]:
execute_sql_statement("SELECT * FROM ChargeDetails;", conn)

[(1,
  'Homicide',
  '187(A)PC:192(C)(1)PC:191.5(A)PC:',
  'MURDER:FIRST DEGREE:SHOOT FROM VEHICLE:VEHICULAR MANSLAUGHTER W/GROSS NEGLIGENCE:VEH MANSLAUGHTER DUI W/GROSS NEGLIGENCE:'),
 (2,
  'Rape',
  '261(A)(1)PC:286(D)PC:288A(A)PC:261(A)(4)PC:261(A)(2)PC:A261(A)1PC:288A(B)1PC:',
  'RAPE - VICTIM INCAPABLE OF CONSENT:SODOMY ACTING IN CONCERT W/FORCE:ORAL COPULATION:RAPE -VICTIM UNCONSCIOUS OF NATURE OF ACT:RAPE BY FORCE/FEAR:ATTEMPT RAPE VICTIM INCAPABLE OF CONSENT:ORAL COPULATION W/PERSON UNDER 18 YRS:'),
 (3,
  'Robbery',
  '211PC:215PC:A211PC:215(A)PC:A215(A)PC:',
  'ROBBERY:CARJACKING:ATTEMPT - ROBBERY:CARJACKING:ATTEMPTED CARJACKING:'),
 (4,
  'Aggravated Assault',
  '273.5(A)PC:245(A)(1)PC:245(A)(2)PC:243(D)PC:451(B)PC:245(C)PC:A273.5(A)PC:243(C)PC:A187(A)PC:273A(A)PC:23110(B)VC:417(A)(2)PC:417.4PC:368(A)PC:245(B)PC:368(B)(1)PC:273D(A)PC:148(A)(1)PC:273.5(E)PC:206PC:245(A)PC:',
  'CORPORAL INJURY ON SPOUSE/COHABITANT/ETC:ADW, NOT FIREARM, W/GBI:ADW W/FIREARM:BATTERY W/GBI:ARSON

In [ ]:
sql_location_detals = '''CREATE TABLE LocationDetails(AreaID INTEGER NOT NULL PRIMARY KEY,
                                                    AreaName TEXT NOT NULL,
                                                    ReportingDistrict INTEGER NOT NULL,
                                                    Address TEXT NOT NULL,
                                                    CrossStreet TEXT NOT NULL,
                                                    Location TEXT NOT NULL
                                                    );"


'''
create_table(conn, sql_location_detals)


sql = """INSERT INTO LocationDetails VALUES (?,?,?,?,?,?);"""
rows = [(1,"Test",1, "Test","Test","Test")]
cur = conn.cursor()
cur.executemany(sql, rows)
execute_sql_statement("SELECT * FROM LocationDetails;", conn)

[(1, 'Test', 1, 'Test', 'Test', 'Test')]

In [ ]:


sql_arrest = '''create table ArrestDetails(ReportID INTEGER NOT NULL PRIMARY KEY, 
                                        ReportType TEXT NOT NULL, 
                                        ArrestDate TEXT NOT NULL,
                                        ArrestTime INTEGER NOT NULL, 
                                        ArrestTypeCode TEXT NOT NULL,
                                        Age INTEGER NOT NULL,
                                        SexCode TEXT NOT NULL,
                                        DescentCode TEXT NOT NULL,
                                        AreaID INTEGER NOT NULL,
                                        ChargeGroupCode INTEGER NOT NULL,
                                        FOREIGN KEY (AreaID) REFERENCES LocationDetails(AreaID),
                                        FOREIGN KEY (ChargeGroupCode) REFERENCES ChargeDetails(ChargeGroupCode)
                                        );
'''
create_table(conn, sql_arrest)

sql_st = """INSERT INTO ArrestDetails VALUES (?,?,?,?,?,?,?,?,?,?);"""

records = [(45,"RFC","12-11-2021",21,"FD",22,"M","AS",1,1)]
cur = conn.cursor()
cur.executemany(sql_st, records)

In [ ]:
execute_sql_statement("SELECT * FROM ArrestDetails;", conn)

[(45, 'RFC', '12-11-2021', 21, 'FD', 22, 'M', 'AS', 1, 1)]